In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

from time import time
from icecream import ic

from msfm.utils import analysis, parameters
from msfm import fiducial_pipeline

import matplotlib.pyplot as plt

### strategy

In [3]:
strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.MirroredStrategy(['/gpu:0', '/gpu:1'])
# strategy = tf.distribute.get_strategy()

print(tf.config.list_physical_devices("GPU"))
print(strategy.num_replicas_in_sync)

n_gpus = strategy.num_replicas_in_sync

2023-03-03 06:59:20.520476: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 06:59:22.272079: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38277 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:03:00.0, compute capability: 8.0
2023-03-03 06:59:22.273272: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38277 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:41:00.0, compute capability: 8.0
2023-03-03 06:59:22.274430: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/devi

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]
4


# simple example

In [4]:
strategy = tf.distribute.MirroredStrategy()
n_replicas = strategy.num_replicas_in_sync

global_batch_size = 8
local_batch_size = global_batch_size//n_replicas

ic(n_replicas)
ic(global_batch_size)
ic(local_batch_size);

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


ic| n_replicas: 4
ic| global_batch_size: 8
ic| local_batch_size: 2


# the different workers all generate different random numbers
> Stateful dataset transformations are currently not supported with tf.distribute and any stateful ops that the dataset may have are currently ignored. For example, if your dataset has a map_fn that uses ?tf.random.uniform to rotate an image, then you have a dataset graph that depends on state (i.e the random seed) on the local machine where the python process is being executed.
- from https://www.tensorflow.org/tutorials/distribute/input#caveats

I don't understand what the above means. It looks like the random numbers are not being ignored

In [8]:
dist = tfp.distributions.MultivariateNormalDiag(loc=[0,0,0], scale_diag=[1,2,3])

def dset_sample(element):
    # sample = dist.sample()
    sample = tf.random.uniform((1,), seed=1)
    # tf.print(sample)
    
    element *= sample
    
    return element

In [9]:
def dataset_fn(input_context):
    dset = tf.data.Dataset.from_tensors([1.0, 1.0, 1.0]).repeat(10)
    dset = dset.shard(input_context.num_input_pipelines, input_context.input_pipeline_id)
    dset = dset.map(dset_sample)
    
    dset = dset.batch(local_batch_size)
    
    return dset

dist_dset = strategy.distribute_datasets_from_function(dataset_fn)

i = 0
for x in dist_dset:
    print(i)
    print(x, "\n")
    
    i += 1

0
PerReplica:{
  0: tf.Tensor(
[[0.2390374  0.2390374  0.2390374 ]
 [0.22267115 0.22267115 0.22267115]], shape=(2, 3), dtype=float32),
  1: tf.Tensor(
[[0.34304976 0.34304976 0.34304976]
 [0.8514762  0.8514762  0.8514762 ]], shape=(2, 3), dtype=float32),
  2: tf.Tensor(
[[0.97840536 0.97840536 0.97840536]
 [0.93370306 0.93370306 0.93370306]], shape=(2, 3), dtype=float32),
  3: tf.Tensor(
[[0.76375294 0.76375294 0.76375294]
 [0.03848493 0.03848493 0.03848493]], shape=(2, 3), dtype=float32)
} 

1
PerReplica:{
  0: tf.Tensor(
[[0.2556386 0.2556386 0.2556386]
 [0.4186945 0.4186945 0.4186945]], shape=(2, 3), dtype=float32),
  1: tf.Tensor([], shape=(0, 3), dtype=float32),
  2: tf.Tensor([], shape=(0, 3), dtype=float32),
  3: tf.Tensor([], shape=(0, 3), dtype=float32)
} 



# determinism with stateless distributions?

In [15]:
def dset_sample(element):
    # sample = dist.sample()
    sample = tf.random.stateless_uniform(((1,)), seed=1)
    # tf.print(sample)
    
    element *= sample
    
    return element

In [16]:
def dataset_fn(input_context):
    dset = tf.data.Dataset.from_tensors([1.0, 1.0, 1.0]).repeat(10)
    dset = dset.shard(input_context.num_input_pipelines, input_context.input_pipeline_id)
    dset = dset.map(dset_sample)
    
    dset = dset.batch(local_batch_size)
    
    return dset

dist_dset = strategy.distribute_datasets_from_function(dataset_fn)

i = 0
for x in dist_dset:
    print(i)
    print(x, "\n")
    
    i += 1

ValueError: in user code:

    File "/tmp/ipykernel_4912/2046651987.py", line 3, in dset_sample  *
        sample = tf.random.stateless_uniform((1,), seed=1)

    ValueError: Shape must be rank 1 but is rank 0 for '{{node stateless_random_uniform/StatelessRandomGetKeyCounter}} = StatelessRandomGetKeyCounter[Tseed=DT_INT32](stateless_random_uniform/StatelessRandomGetKeyCounter/seed)' with input shapes: [].


In [29]:
tf.random.stateless_uniform(shape=[], seed=[1, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.09827709>

In [21]:
tf.random.uniform(shape=[], seed=1)

<tf.Tensor: shape=(), dtype=float32, numpy=0.22267115>